# Devoir maison pour le compte du cours de DBS

In [ ]:
#!pip install pandas rdflib
#!pip install SPARQLWrapper

In [ ]:
# Import la librarie rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace, XSD

## Section 0 :  Choix des jeux de données

### Les sources de données

#### 1) NBA games data, Nathan Lugan (Kaggle)
Link : https://www.kaggle.com/datasets/nathanlauga/nba-games?select=games.csv

Colonnes d'intéret : 

GAME_DATE_EST : Le jour du match

GAME_ID : L'identifiant du match

HOME_TEAM_ID : L'identifiant de l'équipe à domicile

VISITOR_TEAM_ID : L'identifiant de l'équipe à l'exterieur

SEASON : La saison

PTS_home : Nombre de points dans le match de l'équipe à domicile

PTS_away : Nombre de points dans le match de l'équipe à l'exterieur

HOME_TEAM_WINS : Valeur booléenne qui si l'équipe à domicile à gagné le match (1) ou non (0)

#### 2) NBA Database, Wyatt Walsh (Kaggle). Lien : https://www.kaggle.com/datasets/wyattowalsh/basketball

person_id : Identifiant de chaque joueur

first_name : Prénom du joueur

last_name : Nom du joueur

birthdate : Date d'anniversaire

school : L'université de laquelle le joueur vient

height : La taille du joueur

position : Sa position

team_id : L'identifiant de son équipe

team_name : Le nom de son équipe

team_city : Ville de son équipe

team_abbreviation : L'abréviation de son équipe

draft_year : L'année de draft du joueur

draft_round : Le tour auquel le joueur a été drafté



# Section 1 : Préprocessing des données

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None) # Afficher toutes les lignes
pd.set_option("display.max_columns", None) # Afficher toutes les colonnes

In [ ]:
# Import the dataset of players

players = pd.read_csv("./data/Players.csv")
players = players[["person_id", "first_name", "last_name", "birthdate", "school", "height", "position", "team_id", "team_name", "team_city", "team_abbreviation", "draft_year", "draft_round"]]
players.info()

In [ ]:
# Import the Dataset games
games = pd.read_csv("./data/Games.csv")
games = games[["GAME_DATE_EST", "GAME_ID", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "PTS_home", "PTS_away", "HOME_TEAM_WINS"]]
games.info()

In [ ]:
games.dropna(inplace=True)
games.info()

In [ ]:
players.dropna(inplace=True)
players.info()


In [ ]:
players["team_id"].nunique()    

In [ ]:
# Sets de team_id présents dans games
games_teams = set(games["HOME_TEAM_ID"].unique()).union(set(games["VISITOR_TEAM_ID"].unique()))

# Filtrer players pour ne garder que les team_id présents dans games
players_cleaned = players[players["team_id"].isin(games_teams)]

# Remplacer les valeurs "undrafted" par None dans la colonne draft_round
players_cleaned["draft_round"] = players_cleaned["draft_round"].replace("Undrafted", "-1")

# Convertir la colonne birthday en datetime
players_cleaned["birthdate"] = pd.to_datetime(players_cleaned["birthdate"])


# Sauvegarder si besoin
players_cleaned.to_csv("./data/Players_cleaned.csv", index=False)

In [ ]:
players = pd.read_csv("./data/Players_cleaned.csv")

In [ ]:
# Extraire les colonnes équipe depuis player.csv
teams_from_players = players[["team_id", "team_name", "team_city", "team_abbreviation"]].drop_duplicates()

# Extraire les équipes présentes dans game.csv
home_teams = games[["HOME_TEAM_ID"]].rename(columns={"HOME_TEAM_ID": "team_id"})
visitor_teams = games[["VISITOR_TEAM_ID"]].rename(columns={"VISITOR_TEAM_ID": "team_id"})
teams_from_games = pd.concat([home_teams, visitor_teams]).drop_duplicates()

# Fusionner pour avoir toutes les équipes
teams = pd.merge(teams_from_games, teams_from_players, on="team_id", how="left")

# Sauvegarder en CSV
teams_unique = teams.groupby("team_id").agg(lambda x: x.mode()[0]).reset_index()
teams_unique.to_csv("./data/Team_cleaned.csv", index=False)


In [ ]:
games.columns

In [ ]:
games.isna().sum() 

In [ ]:
players.isna().sum()

In [ ]:
games.to_csv("./data/Games_cleaned.csv", index=False)
players.to_csv("./data/Players_cleaned.csv", index=False)

# Section 2 : Conversion de CSV à RDF

L'objectif de cette partie est de definir un graphe RDF noté g, qui va capturer toutes les données de nos tables csv Games, Players et Teams qui seront représentées en RDF comme des classes ayant des propriétés.

Chaque individu aura un identifiant unique appelé URI formé de l'url du namespace et de son id unique, et des valeurs des propriétés de sa classe

In [ ]:
games.info()
games.head()

In [ ]:
players.info()
players.head()

In [ ]:
#players["birthdate"] = pd.to_datetime(players["birthdate"])

In [ ]:
teams_unique.info()

In [ ]:
# Definir un namespace pour notre graphe
NBA = Namespace("https://example.org/nba/")

# Créer un graphe RDF
g = Graph()

# Ajouter des namespaces au graphe
g.bind("nba", NBA)
g.bind("rdf", RDF)
g.bind("xsd", XSD)

# Ajouter des équipes au graphe
for _, row in teams_unique.iterrows():
    # Definir l'URI de l'équipe
    teams_uri = URIRef(NBA[f"{row["team_id"]}"]) 
    
    # Ajouter le type de l'équipe
    g.add((teams_uri, RDF.type, NBA.Team)) 
    
    # Ajouter les propriétés de l'équipe
    g.add((teams_uri, NBA.teamName, Literal(row["team_name"], datatype=XSD.string))) # Ajouter le nom de l'équipe
    g.add((teams_uri, NBA.teamCity, Literal(row["team_city"], datatype=XSD.string))) # Ajouter la ville de l'équipe
    g.add((teams_uri, NBA.teamAbbreviation, Literal(row["team_abbreviation"], datatype=XSD.string))) # Ajouter l'abréviation de l'équipe
    

In [ ]:
# Ajout des joueurs au graphe
for _, row in players.iterrows():
    # Definir l'URI du joueur
    player_uri = URIRef(NBA[f"{row["person_id"]}"])
    
    # Ajouter le type du joueur
    g.add((player_uri, RDF.type, NBA.Player))
    
    # Ajouter les propriétés du joeur
    g.add((player_uri, NBA.firstName, Literal(row["first_name"], datatype=XSD.string))) # Ajouter le prénom du joueur
    g.add((player_uri, NBA.lastName, Literal(row["last_name"], datatype=XSD.string))) # Ajouter le nom du joueur
    g.add((player_uri, NBA.birthdate, Literal(row["birthdate"], datatype=XSD.date))) # Ajouter la date de naissance du joueur
    g.add((player_uri, NBA.school, Literal(row["school"], datatype=XSD.string))) # Ajouter l'école du joueur
    g.add((player_uri, NBA.height, Literal(row["height"], datatype=XSD.string))) # Ajouter la taille du joueur
    g.add((player_uri, NBA.position, Literal(row["position"], datatype=XSD.string))) # Ajouter la position du joueur
    g.add((player_uri, NBA.draftYear, Literal(row["draft_year"], datatype=XSD.gYear))) # Ajouter l'année de draft du joueur
    g.add((player_uri, NBA.draftRound, Literal(row["draft_round"], datatype=XSD.integer))) # Ajouter le round de draft du joueur
    g.add((player_uri, NBA.playsFor, URIRef(NBA[f"{row['team_id']}"]))) # Lier le joueur à son équipe
    
    

In [ ]:
# Ajout the matchs au graphe
for _, row in games.iterrows():
    # Definir l'URI du match
    game_uri = URIRef(NBA[f"{row["GAME_ID"]}"])
    
    # Ajouter le type du match
    g.add((game_uri, RDF.type, NBA.Game))
    
    # Ajouter les propriétés du match
    g.add((game_uri, NBA.gameDate, Literal(row["GAME_DATE_EST"], datatype=XSD.date))) # Ajouter la date du match
    g.add((game_uri, NBA.season, Literal(row["SEASON"], datatype=XSD.gYear))) # Ajouter la saison du match
    g.add((game_uri, NBA.homeTeam, URIRef(NBA[f"{row["HOME_TEAM_ID"]}"]))) # Ajouter l'équipe à domicile
    g.add((game_uri, NBA.visitorTeam, URIRef(NBA[f"{row["VISITOR_TEAM_ID"]}"]))) # Ajouter l'équipe à l'extérieur
    g.add((game_uri, NBA.ptsHome, Literal(row["PTS_home"], datatype=XSD.integer))) # Ajouter les points de l'équipe à domicile
    g.add((game_uri, NBA.ptsAway, Literal(row["PTS_away"], datatype=XSD.integer))) # Ajouter les points de l'équipe à l'extérieur
    g.add((game_uri, NBA.homeTeamWins, Literal(row["HOME_TEAM_WINS"], datatype=XSD.boolean))) # Ajouter si l'équipe à domicile a gagné
    

In [ ]:
# Sauvegarder le graphe RDF dans un fichier Turtle
g.serialize(destination='./turtles_files/nba_graph.ttl', format='turtle')
print("RDF intégré sauvegardé dans nba_graph.ttl")

# Section 3 : REQUETES SPARQL

Le but de cette partie est d'extraire du graphe nouvellement crée certaines informations sur nos données grace à des requetes Sparql

## Requete 1 : Afficher les pages wikipedia des équipes en fonction de leur nombre de points lors de la saison 2018

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from rdflib import Graph

# Initialize your graph
g.parse("./turtles_files/nba_graph.ttl", format="turtle")

# Use SPARQLWrapper instead of direct SERVICE clause
query_local = """
PREFIX nba: <https://example.org/nba/>
SELECT ?teamName ?maxPoints
WHERE {
  {
    SELECT ?team (MAX(?points) AS ?maxPoints)
    WHERE {
      ?game nba:gameDate ?date ;
            (nba:homeTeam | nba:visitorTeam) ?team ;
            (nba:ptsHome | nba:ptsAway) ?points .
      FILTER(YEAR(?date) = 2018)
    }
    GROUP BY ?team
  
  }
  ?team nba:teamName ?teamName .
}
ORDER BY DESC(?maxPoints)

"""

# Execute local query first
results_local = g.query(query_local)

# Then query DBpedia separately
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

results1 = []

for row in results_local:
    team_name = str(row.teamName)
    
    # Query DBpedia for this team
    dbpedia_query = """
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    SELECT ?dbpTeam ?wikiPage
    WHERE {
        ?dbpTeam foaf:name "%s"@en ;
                 foaf:isPrimaryTopicOf ?wikiPage .
    }
    """ % team_name.replace('"', '\\"')
    
    sparql.setQuery(dbpedia_query)
    try:
        dbpedia_results = sparql.query().convert()
        for dbp_row in dbpedia_results["results"]["bindings"]:
            results1.append({
                "teamName": team_name,
                "maxPoints": row.maxPoints,
                "wikiPage": dbp_row["wikiPage"]["value"]
            })
    except Exception as e:
        print(f"Error querying DBpedia for {team_name}: {e}")

In [ ]:
print(pd.DataFrame(results1))

## Requete 2 : Afficher les joueurs qui jouent au poste de center et qui ont une équipe ou non

In [ ]:
g.parse("./turtles_files/nba_graph.ttl", format="turtle")
query2 = """
SELECT ?firstname ?lastname ?playerPosition ?teamName
WHERE {
    {
        SELECT ?firstname ?lastname ?teamID ?playerPosition
        WHERE {
            ?player nba:firstName ?firstname ; 
                    nba:lastName ?lastname ; 
                    nba:position ?playerPosition ; 
                    nba:playsFor ?teamID .
            FILTER(?playerPosition="Center")
        }
    }
   OPTIONAL{?teamID nba:teamName ?teamName} .
}
"""
results2 = g.query(query2)


In [ ]:
# Show the results
for row in results2:
    print(f"{row.firstname} {row.lastname} - Position: {row.playerPosition} - Team: {row.teamName}")

## Requete 3 : Afficher tous les joueurs qui n'ont pas étudié à l'université de californie

In [ ]:
query3 = """
PREFIX nba: <https://example.org/nba/>

SELECT ?firstname ?lastname ?school
WHERE {
  ?player nba:firstName ?firstname ;
          nba:lastName ?lastname ;
          nba:school ?school .

  FILTER ( !CONTAINS(LCASE(STR(?school)), "chicago") )
}

"""
results3 = g.query(query3)

In [ ]:
# Show the results
for row in results3:
    print(f"{row.firstname} {row.lastname} - School: {row.school}")

## Requete 4 : Afficher les équipes qui n'ont pas pour villes : Utah et New Orleans (résultat attendu: 28)

In [ ]:
query4="""
PREFIX nba: <https://example.org/nba/>

SELECT DISTINCT ?teamName ?teamCity
WHERE {
  ?team nba:teamName ?teamName ;
        nba:teamCity ?teamCity .

  MINUS { ?team nba:teamCity "Utah" . }
  MINUS { ?team nba:teamCity "New Orleans" . }
}

"""
results4 = g.query(query4)

In [ ]:
# Show the results
for row in results4:
    print(f"Team Name : {row.teamName} - Team City: {row.teamCity}")

# Requete 5 : Afficher les équipes qui ont joué plus de 82 matchs lors de la saison 2020

In [ ]:
query5 = """
PREFIX nba: <https://example.org/nba/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?team ?teamName (COUNT(?game) AS ?nbMatchs)
WHERE {
  ?game (nba:homeTeam | nba:visitorTeam) ?team ;
        nba:season ?seasonRaw .

  # Conversion de la saison en entier
  BIND(STRDT(STR(?seasonRaw), xsd:integer) AS ?season)

  FILTER(?season = 2020)

  ?team nba:teamName ?teamName .
}
GROUP BY ?team ?teamName
HAVING (COUNT(?game) > 82)
ORDER BY DESC(?nbMatchs)
"""
results5 = g.query(query5)

In [ ]:
# Show the results
for row in results5:
    print(f"Team ID : {row.team} - Team Name : {row.teamName} - Number of game: {row.nbMatchs}")

# Requete 6 : Matchs où l'équipe à domicile a gagné

In [ ]:
query6 = """
PREFIX nba: <https://example.org/nba/>

SELECT ?game ?w
WHERE {
  ?game a nba:Game ;
        nba:homeTeamWins ?w .
  FILTER( STR(?w) = "1")
}
LIMIT 20


"""
results6 = g.query(query6)
for row in results6:
    print(f"Game ID : {row.game} - Home Team Wins : {row.w}")


# Requête 7: faire un classement des équipes sur une saison à partir des matchs, en calculant des indicateurs statistiques avancés.

In [ ]:
query7 = """
PREFIX nba: <https://example.org/nba/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teamName
       (COUNT(?game) AS ?gamesPlayed)
       (SUM(?win) AS ?wins)
       ((SUM(?win) * 100.0) / COUNT(?game) AS ?winPct)
       (AVG(?pf) AS ?avgPointsFor)
       (AVG(?pa) AS ?avgPointsAgainst)
       ((AVG(?pf) - AVG(?pa)) AS ?avgPointDiff)
WHERE {
  {
    # Cas 1 : équipe à domicile
    ?game a nba:Game ;
          nba:season ?season ;
          nba:homeTeam ?team ;
          nba:ptsHome ?pf ;
          nba:ptsAway ?pa ;
          nba:homeTeamWins ?w .
    FILTER(STR(?season) = "2020")
    BIND(xsd:integer(?w) AS ?win)
  }
  UNION
  {
    # Cas 2 : équipe à l'extérieur
    ?game a nba:Game ;
          nba:season ?season ;
          nba:visitorTeam ?team ;
          nba:ptsAway ?pf ;
          nba:ptsHome ?pa ;
          nba:homeTeamWins ?w .
    FILTER(STR(?season) = "2020")
    BIND(1 - xsd:integer(?w) AS ?win)
  }

  ?team nba:teamName ?teamName .
}
GROUP BY ?teamName
HAVING (COUNT(?game) >= 50)
ORDER BY DESC(?winPct) DESC(?avgPointDiff)
LIMIT 5




""" 

In [ ]:
results7 = g.query(query7)

print("Team | Games | Wins | Win% | Avg PF | Avg PA | Avg Diff")
print("-" * 65)

for row in results7:
    teamName = row.teamName
    games    = int(row.gamesPlayed)
    wins     = int(row.wins)
    winPct   = float(row.winPct)
    avgPF    = float(row.avgPointsFor)
    avgPA    = float(row.avgPointsAgainst)
    avgDiff  = float(row.avgPointDiff)

    print(f"{teamName:5} {games:5} {wins:5} {winPct:6.1f}% {avgPF:8.1f} {avgPA:8.1f} {avgDiff:8.1f}")


## Requete 8: Requete renvoyant les équipes ayant participé à aumoins un match que ce soit à domicile ou à l'extérieur(property path |)

In [ ]:
query8 = """
PREFIX nba: <https://example.org/nba/>

SELECT DISTINCT ?team ?teamName
WHERE {
  ?game a nba:Game ;
        (nba:homeTeam | nba:visitorTeam) ?team .
  ?team nba:teamName ?teamName .
}
ORDER BY ?teamName

""" 
result8 = g.query(query8)
for row in result8:
    print(f"Team ID : {row.team} - Team Name : {row.teamName}")

# Requête 9: faire une requête sur des grqphes nommés

In [ ]:
from rdflib import Dataset
from owlrl import RDFSClosure
from owlrl.AxiomaticTriples import RDFS_Axiomatic_Triples, RDFS_D_Axiomatic_Triples

DATA_TTL = "./turtles_files/nba_graph.ttl"
SCHEMA_TTL = "./turtles_files/schema_rdfs_complet.ttl"

ds = Dataset()

g_base = ds.graph("urn:graph:base")   # graphe nommé BASE
g_inf  = ds.graph("urn:graph:inf")    # graphe nommé INF

# Charger BASE
g_base.parse(DATA_TTL, format="turtle")
g_base.parse(SCHEMA_TTL, format="turtle")

# Copier BASE -> INF puis inférer
for t in g_base:
    g_inf.add(t)

RDFSClosure.RDFS_Semantics(
    g_inf,
    RDFS_Axiomatic_Triples,
    RDFS_D_Axiomatic_Triples
).closure()

print("Triples BASE:", len(g_base))
print("Triples INF :", len(g_inf))
print("Ajoutés     :", len(g_inf) - len(g_base))


In [ ]:
# Lister les graphes nommés présents dans le schéma
q_list = """
SELECT DISTINCT ?g
WHERE { GRAPH ?g { ?s ?p ?o } }
ORDER BY ?g
"""
for row in ds.query(q_list):
    print(row.g)


# Requête 10: compter le nombre de relations nba:involvesTeam dans chaque graphe (0 attendu dans base)

In [ ]:
q_compare = """
PREFIX nba: <https://example.org/nba/>

SELECT ?source (COUNT(*) AS ?n)
WHERE {
  VALUES (?source ?gname) {
    ("BASE" <urn:graph:base>)
    ("INF"  <urn:graph:inf>)
  }
  GRAPH ?gname {
    ?game a nba:Game ;
          nba:involvesTeam ?team .
  }
}
GROUP BY ?source
"""
for row in ds.query(q_compare):
    print(row.source, row.n)


# Sesseion 4 : Inférence

Schéma dans schema_rdfs_complet.ttl et inférence avec la librairie owlrl

In [ ]:
!pip install owlrl


In [ ]:
from rdflib import Graph
from owlrl import RDFSClosure
from owlrl.AxiomaticTriples import RDFS_Axiomatic_Triples, RDFS_D_Axiomatic_Triples

DATA_TTL = "./turtles_files/nba_graph.ttl"
SCHEMA_TTL = "./turtles_files/schema_rdfs_complet.ttl"

 
g_base = Graph()
g_base.parse(DATA_TTL, format="turtle")
g_base.parse(SCHEMA_TTL, format="turtle")

 
g_inf = Graph()
for t in g_base:
    g_inf.add(t)

RDFSClosure.RDFS_Semantics(g_inf, RDFS_Axiomatic_Triples, RDFS_D_Axiomatic_Triples).closure()





# Vérifier qu'il y a de nouveaux triplets 

In [ ]:
print("Triples RAW:", len(g_base))
print("Triples INF:", len(g_inf))
print("Triples ajoutés:", len(g_inf) - len(g_base))

# Tester les inférences via des requêtes

In [ ]:
def run_query(title: str, query: str, limit_rows: int = 10):
    print(f"\n===== {title} =====")
    print("---- base ----")
    raw_res = list(g_base.query(query))
    if not raw_res:
        print("(aucun résultat)")
    else:
        for i, row in enumerate(raw_res[:limit_rows], start=1):
            print(f"{i}. {row}")
        if len(raw_res) > limit_rows:
            print(f"... ({len(raw_res)} lignes au total)")

    print("---- INF ----")
    inf_res = list(g_inf.query(query))
    if not inf_res:
        print("(aucun résultat)")
    else:
        for i, row in enumerate(inf_res[:limit_rows], start=1):
            print(f"{i}. {row}")
        if len(inf_res) > limit_rows:
            print(f"... ({len(inf_res)} lignes au total)")

 

queries = [
    (
        "Q1 - subPropertyOf : compter involvesTeam (attendu RAW=0, INF>0)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT (COUNT(*) AS ?n)
        WHERE {
          ?g a nba:Game ;
             nba:involvesTeam ?t .
        }
        """,
        10
    ),
    (
        "Q2 - subPropertyOf : afficher quelques triples inférés involvesTeam",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT ?game ?team
        WHERE {
          ?game a nba:Game ;
                nba:involvesTeam ?team .
        }
        LIMIT 10
        """,
        10
    ),
    (
        "Q3 - subClassOf : compter les Person (Player => Person)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT (COUNT(DISTINCT ?p) AS ?n)
        WHERE { ?p a nba:Person . }
        """,
        10
    ),
    (
        "Q4 - subClassOf : joueurs qui sont à la fois Player et Person (preuve)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT ?player
        WHERE {
          ?player a nba:Player ;
                  a nba:Person .
        }
        LIMIT 10
        """,
        10
    ),
    (
        "Q5 - domain : playsFor => Player (si domain défini sur playsFor)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT (COUNT(DISTINCT ?p) AS ?n)
        WHERE {
          ?p nba:playsFor ?t .
          ?p a nba:Player .
        }
        """,
        10
    ),
    (
        "Q6 - range : playsFor => Team (si range défini sur playsFor)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT (COUNT(DISTINCT ?t) AS ?n)
        WHERE {
          ?p nba:playsFor ?t .
          ?t a nba:Team .
        }
        """,
        10
    ),
    (
        "Q7 - combiné : équipes impliquées dans des matchs (via involvesTeam inféré)",
        """
        PREFIX nba: <https://example.org/nba/>
        SELECT DISTINCT ?team
        WHERE {
          ?game a nba:Game ;
                nba:involvesTeam ?team .
        }
        LIMIT 10
        """,
        10
    ),
]
 
for title, q, lim in queries:
    run_query(title, q, limit_rows=lim)

 
